In [ ]:
include("../src/Apophis.jl")

In [ ]:
using .Apophis

In [ ]:
# using TaylorIntegration, LinearAlgebra # Apophis.jl @reexports TaylorIntegration and LinearAlgebra
using Plots, JLD, DelimitedFiles, Dates
using Statistics: mean, std

In [ ]:
vars = ["tv_neodys_obs", "xv1", "tvS1", "xvS1", "gvS1"] #names of variables
for i in eachindex(vars)
    filename = string("Apophis_jt_", vars[i], ".jld")
    filename
    ex = Symbol(vars[i])
    @eval $ex = load($filename, vars[$i])

    #read solution from files and assign recovered variable to recovered_sol_i
    # recovered_sol_i = load(filename, vars[i])

    #check that solution was recovered succesfully
    # @show recovered_sol_i == sol[i]
end

In [ ]:
tv_neodys_obs#, xv1, tvS1, xvS1, gvS1

In [ ]:
xv1

In [ ]:
apophisdofs = union(34:36, 70:72)
ssdofs = setdiff(1:72, apophisdofs)
sort(union(apophisdofs, ssdofs)) == collect(1:72)

In [ ]:
xv1[2,apophisdofs]

In [ ]:
xv1[20,ssdofs[end-6:end]]

In [ ]:
# read NEODyS delay observations (range)
del = readdlm("../Apophis_NEODyS_DEL.dat")
#read NEODyS Doppler obs. (range-rate)
dop = readdlm("../Apophis_NEODyS_DOP.dat")
# read NEODyS delay & Doppler
deldop = readdlm("../Apophis_NEODyS.dat");

# retrieve times of delay observations
del_ind = 3:17
jd_del = Dates.datetime2julian.(  Dates.DateTime.( del[del_ind,3].*"T".*del[del_ind,4] )  )

# retrieve times of Doppler observations
dop_ind = 6:29
jd_dop = Dates.datetime2julian.(  Dates.DateTime.( dop[dop_ind,3].*"T".*dop[dop_ind,4] )  );

#check that all times in jd_dop are elements of tv_neodys_obs; do the same for jd_del
#for i in eachindex(jd_dop)
#    @show in(jd_dop[i], tv_neodys_obs)
#end
;

In [ ]:
dop_neodys_obs_ind = findall(x->in(x, jd_dop), tv_neodys_obs);
del_neodys_obs_ind = findall(x->in(x, jd_del), tv_neodys_obs)

# check that all elements in jd_dop/jd_del were found in tv_neeodys_obs
@show length(dop_neodys_obs_ind) == length(jd_dop)
@show length(del_neodys_obs_ind) == length(jd_del)

In [ ]:
const ea = 4 #Earth's index
const N = 12 

# Apophis geocentric range, radial velocity
r(x) = sqrt( (x[3N-2]-x[3ea-2])^2+(x[3N-1]-x[3ea-1])^2+(x[3N]-x[3ea])^2 )
vr(x) = ( (x[3N-2]-x[3ea-2])*(x[6N-2]-x[3(N+ea)-2])+(x[3N-1]-x[3ea-1])*(x[6N-1]-x[3(N+ea)-1])+(x[3N]-x[3ea])*(x[6N]-x[3(N+ea)]) )/r(x)

In [ ]:
# get range, rad vel values from integration

# tv_neodys_obs[del_neodys_obs_ind[1]]
rv1 = Array{Taylor1{Float64}}(undef, length(jd_del))
vrv1 = Array{Taylor1{Float64}}(undef, length(jd_dop))

for i in eachindex(del_neodys_obs_ind)
    rv1[i] = r(  xv1[del_neodys_obs_ind[i], :]  )
end

for i in eachindex(dop_neodys_obs_ind)
    vrv1[i] = vr(  xv1[dop_neodys_obs_ind[i], :]  )
end

In [ ]:
# A2 = 0 residuals
residual_r = (rv1()-del[del_ind,6]/au)#*au
residual_vr = (vrv1()-dop[dop_ind,6]/au)#*au;

In [ ]:
plot(tv_neodys_obs[del_neodys_obs_ind].-t0, rv1())
scatter!(tv_neodys_obs[del_neodys_obs_ind].-t0, del[del_ind,6]/au, yerror=del[del_ind,7]/au, leg=false, marker=:cross)

In [ ]:
scatter(tv_neodys_obs[del_neodys_obs_ind].-t0, residual_r*au, leg=false, marker=:cross) #, yerror=del[del_ind,7]/au)
title!("geocentric range residuals (km) vs t; A2=0")

In [ ]:
scatter(tv_neodys_obs[dop_neodys_obs_ind].-t0, residual_vr*au, leg=false, marker=:cross) #, yerror=dop[dop_ind,7]/au)
title!("geocentric radial velocity residuals (km/d) vs t; A2=0")

In [ ]:
rv1[1] # R(A2) = p(A2)

In [ ]:
rv1[1]-constant_term(rv1[1]) # δR(A2) = R(A2) - R_obs = p(A2) - R_obs = p[1]*A2+p[2]*A2^2+... (p[0]=R_obs)

In [ ]:
inverse( rv1[1]-constant_term(rv1[1]) ) # A2(δR) = q(δR) = q[1]*δR+q[2]*δR^2+... (q[0]=0)

In [ ]:
A2_δr_v = (  inverse.( rv1-rv1() )  ) # vector of A2(δr) polynomials at each delay observation;
A2_δvr_v = (  inverse.( vrv1-vrv1() )  ) # vector of A2(δvr) polynomials at each Doppler observation;

In [ ]:
A2_del_v = map((x,y)->x(y), A2_δr_v, residual_r); # A2(δr) polynomials evaluated at the δr residuals;
A2_dop_v = map((x,y)->x(y), A2_δvr_v, residual_vr); # A2(δvr) polynomials evaluated at the δvr residuals;

In [ ]:
mean(A2_del_v), std(A2_del_v)

In [ ]:
mean(A2_dop_v), std(A2_dop_v)

In [ ]:
scatter(tv_neodys_obs[del_neodys_obs_ind].-t0, A2_del_v, leg=false, marker=:cross)

In [ ]:
scatter(tv_neodys_obs[dop_neodys_obs_ind].-t0, A2_dop_v, leg=false, marker=:cross)

In [ ]:
using IntervalArithmetic

In [ ]:
residual_r_interval = interval.(rv1()-(del[del_ind,6]+del[del_ind,7])/au, rv1()-(del[del_ind,6]-del[del_ind,7])/au)
residual_vr_interval = interval.(vrv1()-(dop[dop_ind,6]+dop[dop_ind,7])/au, vrv1()-(dop[dop_ind,6]-dop[dop_ind,7])/au);

In [ ]:
A2_del_v_interval = map((x,y)->x(y), A2_δr_v, residual_r_interval); # A2(δr) polynomials evaluated at the δr residuals;
A2_dop_v_interval = map((x,y)->x(y), A2_δvr_v, residual_vr_interval); # A2(δvr) polynomials evaluated at the δvr residuals;

In [ ]:
scatter(tv_neodys_obs[del_neodys_obs_ind].-t0, A2_del_v, yerror=radius.(A2_del_v_interval), leg=false, marker=:cross)

In [ ]:
scatter(tv_neodys_obs[dop_neodys_obs_ind].-t0, A2_dop_v, yerror=radius.(A2_dop_v_interval), leg=false, marker=:cross)